In [ ]:
import pandas as pd
import glob
import tldextract
from bs4 import BeautifulSoup
from statistics import mean 

In [ ]:
PATH_TO_DATA_FOLDER = '../../iboeckmann/webtracking_analysis/parsing/data/archive/parsed_news-websites/'
news = pd.DataFrame()
DATA_FILES = ['us_part1/newspaper.json', 'us_part2/newspaper.json', 'uk/newspaper.json']
for filepath in DATA_FILES:
    current_news = pd.read_json(PATH_TO_DATA_FOLDER+filepath, lines=True)
    current_news['country'] = filepath[:2]
    news = news.append(current_news)
    

news.reset_index(inplace=True, drop=True)


news = news[(news['meta_lang'] == 'en') | (news['meta_lang'].isnull())] #select only articles with 'en' meta_lang or with empty
def extract_domain(row):
    domain = row.split('/')[0]
    domain = domain.replace('www.', '')
    return domain

news['domain'] = news['url'].apply(lambda row: extract_domain(row))


In [ ]:
def get_text(soup):
    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()    # rip it out

    # get text
    text = soup.get_text()

    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = '\n'.join(chunk for chunk in chunks if chunk)
    return text

PATH_TO_HTMLS = "../../iboeckmann/webtracking_analysis/parsing/tmp/Netquest/downloads/us/2019-04-08/web_page/"
htmls_dict = {}
for domain in k_top['domain']:
    df = news.loc[news['domain'] == domain]
    if int(us_domains[us_domains['domain'] == domain]['url']) <15:
        n = int(us_domains[us_domains['domain'] == domain]['url'])
        max_count = 1
    else:
        n = 15
        max_count = 3
    htmls_dict[domain] = {'count_not_found': [], 'count_found': [], 'count_contain_text': [], 'text': []}
    count = 0
    while count < max_count:
        htmls_dict[domain]['count_found'].append(0)
        htmls_dict[domain]['count_contain_text'].append(0)
        htmls_dict[domain]['count_not_found'].append(0)
        sample_df = df['htmlfile'].sample(n = n)
        for sample in sample_df:
            try:
                temp = open(PATH_TO_HTMLS+sample, "r").read()
                htmls_dict[domain]['count_found'][count] += 1
                soup = BeautifulSoup(temp)
                text = get_text(soup)
                if len(text) > 2000:
                    htmls_dict[domain]['count_contain_text'][count] += 1
                    htmls_dict[domain]['text'].append(text)
            except:
                htmls_dict[domain]['count_not_found'][count] += 1
          
        count += 1
        
    htmls_dict[domain]['count_found'] = mean(htmls_dict[domain]['count_found'])
    htmls_dict[domain]['count_not_found'] = mean(htmls_dict[domain]['count_not_found'])
    htmls_dict[domain]['count_contain_text'] = mean(htmls_dict[domain]['count_contain_text'])

In [ ]:
htmls_df = pd.DataFrame(htmls_dict).transpose()
htmls_df['found_pages'] = htmls_df.apply(lambda x: 1 - x['count_not_found']/15, axis=1)
htmls_df['text_pages'] = htmls_df.apply(lambda x: x['count_contain_text']/x['count_found'] if x['count_found']!=0 else 0, axis=1)
htmls_df